In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

import torch

import pyciemss
from pyciemss.PetriNetODE.interfaces import (setup_petri_model, reset_model, intervene_petri_model,
                                             sample_petri, calibrate_petri, optimize, load_petri_model)
import mira
from pyciemss.utils import get_tspan

import warnings
warnings.filterwarnings('ignore')


# Manual Lotka Volterra

## Load model

In [2]:
from pyciemss.PetriNetODE.models import LotkaVolterra

lm = LotkaVolterra(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, add_uncertainty=False)
lm

LotkaVolterra()

## Initialize model

In [3]:
manual_lotka_volterra = setup_petri_model(lm, start_time=0.0, start_state=dict(prey_population=1.0, predator_population=1.0))
manual_lotka_volterra

LotkaVolterra()

## Sample from prior

In [4]:
samples = sample_petri(manual_lotka_volterra, [1, 2, 3], 5)
samples

{'prey_population_sol': tensor([[0.5775, 0.4934, 0.5817],
         [0.5775, 0.4934, 0.5817],
         [0.5775, 0.4934, 0.5817],
         [0.5775, 0.4934, 0.5817],
         [0.5775, 0.4934, 0.5817]]),
 'predator_population_sol': tensor([[0.7790, 0.4805, 0.2991],
         [0.7790, 0.4805, 0.2991],
         [0.7790, 0.4805, 0.2991],
         [0.7790, 0.4805, 0.2991],
         [0.7790, 0.4805, 0.2991]])}


# Load lotka volterra  with uncertainty

In [5]:
from pyciemss.PetriNetODE.models import LotkaVolterra

uncertain_lotka_volterra_raw= LotkaVolterra(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0)
lm

LotkaVolterra()

## Initialize uncertain Lotka-volterra model

In [6]:
uncertain_lotka_volterra = setup_petri_model(uncertain_lotka_volterra_raw,
                                             start_time=0.0, 
                                             start_state=dict(
                                                 prey_population=1.0, 
                                                 predator_population=1.0))
uncertain_lotka_volterra

LotkaVolterra()

## Sample from prior

In [7]:
samples = sample_petri(uncertain_lotka_volterra, [1, 2, 3], 5)
samples

{'alpha': tensor([0.6570, 0.7121, 0.6432, 0.6077, 0.7099]),
 'beta': tensor([1.2142, 1.4194, 1.4256, 1.2692, 1.4073]),
 'gamma': tensor([0.9383, 0.9852, 0.9580, 0.9067, 1.0141]),
 'delta': tensor([1.0158, 1.0602, 1.0584, 1.0652, 0.9489]),
 'prey_population_sol': tensor([[0.6020, 0.4886, 0.5361],
         [0.5382, 0.4393, 0.5152],
         [0.4974, 0.3812, 0.4270],
         [0.5302, 0.3896, 0.4051],
         [0.5806, 0.5218, 0.6488]]),
 'predator_population_sol': tensor([[0.8578, 0.5738, 0.3738],
         [0.8079, 0.4961, 0.3034],
         [0.8072, 0.4831, 0.2813],
         [0.8795, 0.5684, 0.3471],
         [0.7371, 0.4429, 0.2764]])}

## Calibrate uncertain Lotka Volterra

In [8]:
fitted_parameters = calibrate_petri(uncertain_lotka_volterra, 
                                data =[(1, dict(prey_population=0.5,
                                        predator_population=0.7))])
fitted_parameters

AutoLowRankMultivariateNormal()

In [9]:
posterior_samples = sample_petri(uncertain_lotka_volterra, 
                                 timepoints=[1,2,3], num_samples=50, 
                           inferred_parameters=fitted_parameters)
posterior_samples

{'alpha': tensor([0.6796, 0.6477, 0.6701, 0.7218, 0.7087, 0.6328, 0.7041, 0.7278, 0.6972,
         0.6320, 0.6285, 0.6298, 0.6052, 0.6053, 0.6535, 0.7297, 0.6239, 0.6258,
         0.6547, 0.6540, 0.7329, 0.6930, 0.6753, 0.7181, 0.6713, 0.7222, 0.6368,
         0.7355, 0.6453, 0.6665, 0.6170, 0.7186, 0.7182, 0.6706, 0.6045, 0.6332,
         0.6652, 0.6349, 0.6591, 0.6663, 0.7094, 0.6855, 0.6532, 0.6727, 0.7126,
         0.6113, 0.7277, 0.7353, 0.6323, 0.6953]),
 'beta': tensor([1.2092, 1.4336, 1.2174, 1.3852, 1.3751, 1.4024, 1.2277, 1.4320, 1.2308,
         1.4309, 1.4102, 1.2296, 1.3513, 1.2453, 1.3468, 1.4444, 1.4201, 1.2638,
         1.4293, 1.2524, 1.2192, 1.3845, 1.3041, 1.4338, 1.2349, 1.3105, 1.2822,
         1.3041, 1.2683, 1.2601, 1.3337, 1.3482, 1.3276, 1.3896, 1.3836, 1.4062,
         1.2360, 1.3213, 1.2685, 1.4179, 1.2688, 1.2125, 1.4394, 1.2308, 1.3665,
         1.4025, 1.4082, 1.3654, 1.3267, 1.3457]),
 'gamma': tensor([1.0143, 1.0504, 0.9182, 0.9873, 1.0148, 1.0911, 1.004

## Intervene on uncertain Lotka Volterra

In [18]:
intervened_lotka_volterra = intervene_petri_model(uncertain_lotka_volterra,
                                                  [(1.5, 'alpha', 1.0)])
intervened_lotka_volterra

LotkaVolterra()

## Sample from intervened model

In [19]:
intervened_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=[1, 2, 3, 4, 5, 6, 7, 8], num_samples = 10
                                  )
intervened_samples

{'alpha': tensor([0.6670, 0.6623, 0.7019, 0.7128, 0.6147, 0.6533, 0.6137, 0.6914, 0.7369,
         0.6240]),
 'beta': tensor([1.2821, 1.3085, 1.3782, 1.4407, 1.3363, 1.2409, 1.3578, 1.4014, 1.4448,
         1.4597]),
 'gamma': tensor([1.0409, 1.0674, 1.0513, 1.0621, 0.9292, 1.0458, 0.9394, 1.0553, 0.9870,
         1.0499]),
 'delta': tensor([0.9999, 0.9684, 0.9476, 1.0931, 0.9296, 1.0504, 0.9138, 0.9702, 0.9111,
         1.0429]),
 'prey_population_sol': tensor([[0.6110, 0.6431, 1.0235, 1.6249, 1.7142, 0.8916, 0.5384, 0.5701],
         [0.6126, 0.6685, 1.1043, 1.7982, 1.8320, 0.8659, 0.5291, 0.5925],
         [0.6018, 0.6489, 1.0590, 1.7483, 1.9223, 0.9526, 0.5430, 0.5715],
         [0.5456, 0.5405, 0.8474, 1.4099, 1.7245, 0.9618, 0.5102, 0.4937],
         [0.5435, 0.5306, 0.8355, 1.4092, 1.8830, 1.2057, 0.5668, 0.4492],
         [0.6118, 0.6322, 0.9893, 1.5360, 1.5895, 0.8515, 0.5326, 0.5687],
         [0.5406, 0.5353, 0.8577, 1.4700, 1.9761, 1.2189, 0.5581, 0.4482],
         [0.5814,